In [0]:
%pip install transformers pymupdf scikit-learn hf_xet

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install torch

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-824d2b7c-ac4a-47ca-a181-b31ed4937231
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
import fitz  # PyMuPDF
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch


In [0]:
# Simulated extracted text (pretend it came from a PDF)
texts = [
    "Machine learning is a field of artificial intelligence that focuses on teaching computers to learn patterns.",
    "Databricks is a cloud platform for big data and machine learning workflows.",
    "Transformers are a type of deep learning model based on self-attention mechanisms.",
    "Retrieval-augmented generation improves LLMs by letting them fetch facts from a knowledge base.",
    "PyTorch is an open-source machine learning framework based on the Torch library."
]


In [0]:
def embed_texts(texts, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()

    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(embedding)

    return np.array(embeddings)


In [0]:
embeddings = embed_texts(texts)

In [0]:
def search_similar_texts(query, texts, embeddings, model_name="sentence-transformers/all-MiniLM-L6-v2", top_k=3):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()

    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        query_embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()

    query_embedding = np.expand_dims(query_embedding, axis=0)

    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_k:][::-1]

    results = [texts[i] for i in top_indices]
    return results


In [0]:
query = "What does Databricks do?"
top_results = search_similar_texts(query, texts, embeddings)

print("Top Retrieved Passages:")
for i, passage in enumerate(top_results):
    print(f"{i+1}. {passage}")
    print("-" * 50)


Top Retrieved Passages:
1. Databricks is a cloud platform for big data and machine learning workflows.
--------------------------------------------------
2. PyTorch is an open-source machine learning framework based on the Torch library.
--------------------------------------------------
3. Machine learning is a field of artificial intelligence that focuses on teaching computers to learn patterns.
--------------------------------------------------


In [0]:
from transformers import pipeline

# Load a small QA model
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

context = " ".join(top_results)

result = qa_pipeline(question=query, context=context)
print("Generated Answer:")
print(result['answer'])


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Generated Answer:
a cloud platform for big data and machine learning workflows
